
# CSCI E-25
## Introduction to Convolutional Neural Networks
### Steve Elston

## 1.0 Introduction to convolutional neural networks

This lesson introduces you to a powerful neural network architecture, known as **convolutional neural networks**. Convolutional neural networks operate by **learning a set of filters**  or **convolution kernels**. Using a process, known as **convolution**, these filters extract a **feature map** from the raw data, an **embedding**. The feature map is a **lower dimensional** map of the raw input features. This map is learned in a supervised machine learning process employing back propagation. You can think of convolutional neural networks as a powerful and flexible **dimensionality reduction** method. 

Convolutional neural networks are used to build feature maps from any type of data with coherency in one or more dimensions. This includes time series data, and text data as one dimensional cases, and image data as a two dimensional case. 

Convolutional neural networks have a reasonably long history. The first known commercial application was for automated processing of check images by LeCun et. al. (1998). For unclear reasons, convolutional neural networks were relegated to specialized applications such as hand writing recognition until Krizhevsky et. al. (2012) used them to decisively win an ImageNet object recognition competition. Curiously, other teams had previously used convolutional neural networks to win competitions, but somehow this was not widely recognized. 

In this lesson you will learn:
1. The basics of convolutional neural network architecture.
2. How convolutional neural networks learn the filters to create the feature map. 
3. Feature extraction for creating the feature map. 
4. Visualization of the feature map and the filters.
5. Commonly used convolutional operator design. 
6. How to use the feature map in supervised learning. 
7. Application of various regularization methods for improving the behavior of learning. 

****
**Note:** The formulation of convolution filters used in deep learning is somewhat idiosyncratic. If you are familiar with the standard approach as applied in say electrical engineering or control engineering you may find the approach used here a bit different. 
****


> You may need to install some specialized packages required to execute the code in this notebook. Uncomment the code in the cell below and execute to install these packages.
> For some environments you may need to install graphviz by following the instructions for your operating system given [here](https://graphviz.gitlab.io/download/). 

In [ ]:
#!pip install pydot
#!pip install graphviz

Execute the code in the cell below to import the required packages to run this notebook.  

In [ ]:
from keras.datasets import mnist
import tensorflow as tf
from tensorflow import convert_to_tensor 
import keras.utils as ku
from keras.utils import plot_model
import keras.models as models
import keras.layers as layers
from keras.optimizers import RMSprop
from keras.optimizers.schedules import ExponentialDecay
from keras import regularizers, callbacks
from keras.layers import Dropout
from keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy
import numpy as np
import numpy.random as nr
import pandas as pd
from math import exp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import rankdata
import sklearn.metrics as metrics
import math

%matplotlib inline

### 1.1 Applying 2-d convolution

Let's review how we can apply convolution operations in two dimensions. Figure 1.1 illustrates a simple example of applying a convolution operator to a 2-d input array. The 2-d input array can be a gray-scale image, for example. 

A 3X3 convolutional operator is illustrated in Figure 1.1. This operator is applied to a small image of dimension of 4X4 pixels. Within the 4X4 image the operator is only applied four times, where the complete operator lies on the image. Convolution where the operator is completely contained within the input sample is sometimes called call **valid convolution**. 

In this case the 4X4 pixel input results in a 2X2 pixel output. You can think of the convolutional operator mapping from the center of the operator input to pixel in the output. Similar to the 1-d case, for a convolution operator with span $s$ in both dimensions the output array will be reduced in each dimension by $\frac{s + 1}{2}$

![](img/2D-Conv.JPG)
<center>Figure 1.1. Simple 2-d convolution operation</center>

For a convolution with the identical span $s$ in both dimensions we can write the convolution operations as: 

$$S(i,j) = (I * K)(i,j) = \sum_{m = {i - a}}^{i + a} \sum_{n = j - a}^{j + a} I(i, j) K(i-m, j-n)$$

Notice that $S$, $I$ and $K$ are all tensors. Given this fact, the above formula is easily extended to higher dimensional problems. 

The convoluational kernel $K$ and the image $I$ are commutative so there is an alternative representation by applying an operation known as **kernel flipping**. Flipped kernel convolution can be represented as follows:

$$s(i,j) = (I * K)(i,j) = \sum_{m = {i - a}}^{i + a} \sum_{n = j - a}^{j + a} I(i-m, j-n) K(i, j)$$


### 1.2 Convolution in higher dimensions

In deep convolutional neural networks the input typically have several **input channels** with the convolution operation resulting in multiple **output channels**. 

So far, we have only looked at single channel convolution. Here are some examples of input tensors with different numbers of input channels:
- A gray-scale image has a single input channel of pixels. 
- A color image typically has 3 input channels, {Red, Green, Blue}.
- A color movie has a large number of input channels. For $n$ images in the video there are $3 * n$ images counting the Red, Green, Blue channels for each image.  

The purpose of the a convolutional neural network is to create a feature map. To create the most general feature map convolution operators need not be restricted to a single input channel. 

Further, the feature map can can many channels, resulting in multiple output channels. Each channel is a set of features extracted from the input tensor. This concept is illustrated in Figure 1.2 below. In this figure a set of convolution operators transform the input tensors to multiple channels of the feature map. Each convolution operator maps to a specific output channel in the feature map. 

![](img/MultiChannel.JPG)
<center>Figure 1.2. Example of multichannel convolution</center>

Let's investigate the case of a 3-d input tensor $V$, a 3-d output tensor $Z$ and 4-d convolution tensor $K$. In this case the convolution equation becomes:

$$Z_{i,j,k} = (V * Z)(i,j,k,l) = \sum_{l} \sum_{m = -a}^{a}
\sum_{n = -a}^{a}
V_{i,j,l} \cdot K_{i-m,j-n,k,l}$$
where,  
$i,j$ are the spatial dimensions,  
$k$ is the index of the output channel,  
$l$ is the index of the input channel,  
$K_{i,j,k,l}$ is the kernel connecting the $l$th channel of the input to the $k$th channel of the output for pixel offsets $i$ and $j$    
$V_{i,j,l}$ is the $i,j$ input pixel offsets from channel $l$ of the input,  
$Z_{i,j,k}$ is the $ i,j$ pixel offsets from channel $k$ of the output,  
$a = \frac{1}{2}(kernel\_span + 1)$, for an odd length kernel.  

Notice that the summation in the equation above is over input channels $l$ as well as the spatial dimensions $m,n$ of the convolutional operator. This operation is applied for each spatial pixel coordinate $i,j$. This generalization allows the convolutional kernel to operate over multiple input channels. This is property is highly desirable since features should be extracted from the entire input tensor. 

The entire forgoing operation is performed for each output channel $k$. Therefore, you can think of the $k$th dimension of the 4-d kernel tensor as indexing multiple 3-d convolution kernels. 

### 1.3 Parameter sharing and sparse interactions

**Parameter sharing** and **sparse interactions** are two powerful aspects of convolutional neural networks. To understand what these terms mean and why they are important, consider the fully connected neural network in Figure 1.3 below.  
![](img/FullyConnected.JPG)
<center>Figure 1.3. A fully connected network</center>

The fully connected neural network has a large number of weights, $5^2 = 25$ in this case. Compare this network to the convolutional layer with $2 \times 2$ operator will have only $2*2 = 4$ weights. For a more realistic case, the fully connected network might have millions of weights. The convolutional network using **weight sharing** across the entire layer. For a $5 \times 5$ CovNet layer there are only $5 * 5 = 25$ weights. Parameter sharing is also referred to as **tied weights**. This is an example of a **sparse interaction**.

The shared nature of the parameters in convolutional neural networks is a significant reduction in complexity. Statistically, such a sparse model is said to **share statistical strength**. This statistical strength results in model weights with less uncertainty and a model less likely to be over-fit.  

The computational efficiency of convolutional neural networks is not just in training. Convolutional neural networks are also efficient for creating **rich feature maps** which lead to high levels of accuracy in production.

### 1.4 Pooling and invariance

To further reduce dimensionality and to make the feature map robust to small shifts in the input, **pooling** is applied following the nonlinear detection stage. Pooling involves applying an aggregation function to patches of the output from the nonlinear detector of a convolution layer. 

Researchers have tried a number of pooling schemes. In practice **max pooling** (Zhou and Chellappa, 1988) has proved to be both robust and versatile. In max pooling the output value is just the maximum of the input values in each patch. 

Unlike convolutional layers, there are no parameters to learn in pooling layers. 

Max pooling has the nice property of being robust to small shifts in the input values. For example, Figure 1.4 shows the response of max pooling to a right sift of a single pixel for an operator with a span of 3. Notice that the response itself is simply shifted one pixel to the right as well. The pattern of maximum values in the feature map (the output) remains the same. 

![](img/ShiftOne.JPG)
<center>Figure 1.4.   
    Example of max pooling responses to one pixel shift right</center>

### 1.5 Stride and tiling

Up until now, we have only moved the convolution operator one pixel at a time. There is no restriction that requires this type of step, however. In fact, a convolutional operator or pooling operator can be moved by several pixels in any direction. This step size is known as the **stride** of the operator. 

Operators with stride greater than one **decimate** the number of pixels in the output. This can be useful when resizing images of different dimensions so they have constant input dimensions for a neural network. 

Using an operator with $stride = span$ is a special case known as **tiling**. Tiled operators are laid out without any overlap. Figure 1.5 shows an example of tiling. In this case, an input of dimensions $6X6$ is tiled by operators with dimension $3X3$ with stride of 3. Only 4 tiles fit on the input in this case, resulting in a $4X4$ output array.

![](img/Tiled.JPG)
<center>**Figure 1.5. Example of a tiled convolution operator**</center>

> **Exercise 4-1:** You will now undertake an exercise to better understand the effects of max pooling. To simplify this exercise you will work with 1-dimensional data and **edge detection** operators. Do the following:   
> 1. Create a function named `max_pool` that returns maximum of the absolute value kernel operator with a stride of 1 over the input series. The kernel span should be an argument to the function, with default value of 3. For simplicity start the operator at the beginning of the series.     
> 2. Create two edge detection convolutional kernels, $kernel1 = [-0.5, 1.0, -0.5]$ and $kernel2 = [0.5, -1.0, 0.5]$.   
> 3. Apply both of the convolution kernels to the `series1` provided using the [numpy.convolve](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html) function. Assign these two results to separate variables.    
> 4. Apply your `max_pool` function to the convolution result series. Assign these two results to different variables.    
> 5. Display the results comparing the original series, the convolution series to the max-pooled convolution series using the `plot_pool` and 'plot_series' functions provided.   

In [ ]:
def plot_series(series, pool, original_series, x_pool, label, offset, ax):    
    x = list(range(series.shape[0]))
    x0 = [x + offset for x in range(len(original_series))]
    ax.plot(x, series, label = 'Detected series')
    ax.plot(x_pool, pool, color = 'red', label = label)
    ax.plot(x0, original_series, color = 'green', ls='--', label = 'Original series')
    ax.set_title('Series of raw values and max pool values')
    ax.legend()
    ax.set_xlabel('Index')
    ax.set_ylabel('Value') 
    
def plot_pool(series1, series2, original_series, pool1, pool2, span=3):
    x = list(range(series1.shape[0]))
    end = series1.shape[0]
    end_pool = pool1.shape[0]
    end_remain = end % end_pool
    pool_stride = int((end_pool - end_remain)/end_pool) + 1
    x_pool = list(range(end_remain, end_pool + end_remain, pool_stride))
    print('length of the input series = ' + str(len(x)))
    print('length of the max pool series = ' + str(len(x_pool)))

    ## Display the results
    fig,ax = plt.subplots(2, 1, figsize=(8,6))
    ## First channel
    plot_series(series1, pool1, original_series, x_pool, 'First channel max pool', (span -1)/2, ax[0])
    ## Second channel
    plot_series(series2, pool2, original_series, x_pool, 'Second channel max pool', (span -1)/2, ax[1])
    

nr.seed(12233)
series1 = np.concatenate((np.zeros((20,)), np.zeros((5,)) + 1.0, np.zeros((5,)) - 1.0, np.zeros((20,))))

def max_pool(series, span=3, stride=1):
    '''Performs simple 1d max pooling with
    an operator over span and stride 1'''
    ## Your code goes below
    out =[]
    len = series.shape[0]
    offset = int((span + 1)/2) #- 1
    print('offset = ' + str(offset))
    stop = len - 1 
    for i in range(offset, stop, stride): 
        out.append(np.max(np.abs(series[i:i+span])))
    return np.array(out)   


kernel1 = np.array([-0.5, 1.0, -0.5])
kernel2 = np.array([0.5, -1.0, 0.5])
result1 = np.convolve(series1, kernel1)
result2 = np.convolve(series1, kernel2)
    
pool1 = max_pool(result1)
pool2 = max_pool(result2)
plot_pool(result1, result2, series1, pool1, pool2)   

> 6. Define and apply new 5-point convolution kernels for the edge detection, $[-0.25, - 0.25, 1.0, -0.25, -0.25]$ and $[0.25, 0.25, -1.0, 0.25, 0.25]$. 
> 7. Apply the max pooling with kernel span of 5 to both of the edge detection series.  
> 8. Display the results.  

In [ ]:
kernel1 = np.array([-0.25, - 0.25, 1.0, -0.25, -0.25])
kernel2 = np.array([0.25, 0.25, -1.0, 0.25, 0.25])
result1 = np.convolve(series1, kernel1)
result2 = np.convolve(series1, kernel2)

span=3
pool1 = max_pool(result1, span = span)
pool2 = max_pool(result2, span = span)
plot_pool(result1, result2, series1, pool1, pool2, span = span)   

> Examine these results and answer these questions in one or a few sentences:   
> 1. How can you explain the invariance of the max-pooling results to the sign of the convolved series?
> 2. How does the max-pooling operator alter edges deteced by the convolution output series?     
> 3. How can you explain the difference in the edge detection series and the max-pooled values between the two kernel spans in terms of the receptive field of the operator?    
> **End of exercise**.   

> **Answers:**    
> 1.     
> 2.     
> 3.     

## 2.0 Convolutional neural networks with Keras

With the foregoing theory and simple examples, let's try constructing and testing a convolutional neural network using Keras. In this case, we will test classification of of the MNIST dataset. The neural network will have the following layers:
1. An input layer for the 28X28 images.
2. A multi-layer convolutional neural network to create a feature map.
3. A fully-connected hidden layer to perform the classification.
4. A output layer to indicate which digit is most likely. 

### 2.1 Preparing the dataset

The preparation of this dataset is follows nearly identical steps to the process followed in an earlier lesson. In summary, three preparation steps are performed:
1. Load the training and test image data and labels.
2. Reshape the image tensors so the neural network views them as a 4-d tensor. For the training images the tensor has a shape of (60000,28,28,1) corresponding to 60000 images of dimension $28 \times 28$ with a single channel (these are gray-scale images). At the same time the pixel values are converted to `float` type in the range $\{ 0,1 \}$.
3. The labels are represented by dummy variables using the Keras [to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) function.

> **Note:** Notice that the channel index is the last of the 4 used for the image tensors. TensorFlow uses a **channel last** format for image tensors, in $image \times height \times width \times channel$, or NHWC format. If you have worked with Torch you will know that this platform uses a channel first format.  

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float')/255
print(train_images.shape)
print(train_images.dtype)

In [ ]:
print(test_images.shape, test_labels.shape)
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float')/255
print(test_images.shape)

In [ ]:
train_labels = ku.to_categorical(train_labels)
print(train_labels[5:,])

In [ ]:
test_labels = ku.to_categorical(test_labels)

Notice the one-hot encoding of the labels. There is one dummy variable (column) for each label value, $\{0,...,9\}$. 

### 2.2 Defining the model. 

It is now time to define the model. 

> Early stopping regularization is an old idea in machine learning. The concept is simple, stop learning at the point the model starts to over-fit. Early stopping measures learning with a single selected validation metric. The early stopping algorithm can be describe as follows, using a validation metric like validation error rate or validation loss rate.    

As a first step, the callbacks required to implement early stopping regularization are defined. The steps to define the [early stopping call backs with Kera](https://keras.io/api/callbacks/early_stopping/) are:    
1. Define a path to an HDF5 file used to save the training history. 
2. The first callback stops the training when the validation loss does not improve within 2 epochs, the patience.  
3. The second callback saves the model weights resulting from the epoch with the best validation loss.  

In [ ]:
## Set up and call-backs for early stopping
patience = 2
filepath = 'my_model_file.keras' # define where the model is saved
def set_callbacks_list(patience, filepath):
    callbacks_temp = [
        callbacks.EarlyStopping(
            monitor = 'val_loss', # Use accuracy to monitor the model
            patience = patience # Stop after patience steps with lower accuracy
         ),
        callbacks.ModelCheckpoint(
            filepath = filepath, # file where the checkpoint is saved
            monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
            save_best_only = True # Only save model if it is the best
         )
        ]
    
    return callbacks_temp
    
callbacks_list = set_callbacks_list(patience, filepath)

> **Exercise 4-2:** You will now use Keras to define the CovNet model named `nn`. This model uses both convolutional layers and max pooling layers. You will define this model by completing the function in the code cell below. You can find [documentation on the Keras convolutional layers here](https://keras.io/layers/convolutional/). You can find [documentaton on Keras pooling layers here](https://keras.io/layers/pooling/).
> The architecture of the model you will create is:
> 1. A 3X3 convolutional layer with 32 output channels using the [Conv2D](https://keras.io/api/layers/convolution_layers/convolution2D/) function, with `activation='relu'`. Recall that this operation creates a feature map with 32 channels from the single gray-scale input channel. This is the input layer of the model, so make sure you define the input shape. 
> 2. A 2X2 max pooling layer using the [MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/) function.
> 3. A dropout layer is used to improve regularization of the model using the [Dropout](https://keras.io/api/layers/regularization_layers/dropout/) function with argument 0.2. 
> 3. A 3X3 convolution layer with 64 output channels, with `activation='relu'`. The 32 channels of the input layer are mapped to a 64 channel feature map. 
> 4. Another 2X2 max pooling layer. 
> 5. A dropout layer to improve regularization of the model with argument 0.2. 
> 6. A final 3X3 convolution layer with 64 output channels, with `activation='relu'`. 
> 7. A dropout layer to improve regularization of the model with argument 0.2. 
> 8. Flatten the 64 channel feature map to a vector.
> 9. A Dense (fully-connected) hidden layer with 64 units is the first classifier layer, with `activation='relu'`. A limited amount of regularization is applied using Keras [layer regularizer l2](https://keras.io/api/layers/regularizers/) as a ` kernel_regularizer`, with hyperparameter 0.1. This layer operates on the flattened feature map. 
> 10. A dropout layer to improve regularization of the model with argument 0.2. 
> 11. The output Dense layer has 10 units with `activation='softmax'` to indicate the classification of the digits.
> 12. A summary of the model is printed.
> 13. The model object is returned. 
> Execute your code and examine the summary of the model. 

In [ ]:
def create_model(): 
    ## Your code goes below




    
    return nn

nn = create_model()

> There is quite bit of interesting information in the summary of this model. In one or a few sentences answer the following questions:
> 1. How does the dimensionality of the feature map change at each convolution and max pooling layer? 
> 2. Compare the number of parameters of your CovNet model to the fully connected model you created for Exercise 3-4. How different is the overall number of parameters and layers? 
> 3. What does the difference in the numbers of parameters between the CovNet and the fully connected model tell you about the ability to train these models with a finite amount of data? 
> 4. How many learnable parameters do the max-pooling layers have, and why?    
> **End of exercise**. 


> **Answers:** 
> 1.            
> 2.           
> 3.           
> 4.              


### 2.3 Train and test the model

Now, it is time to compile and fit your model. This particular model appears to require only limited regularization. The table below shows the limited heuristic search performed on the hyperparameters.

| Final Accuracy  | Dropout  | weight decay | Learning Rate |   
| --------------- | -------- | ------------ | ------------- |
| 0.9923          | 0.2      | 0.00         | 0.001        |
| 0.9923          | 0.2      | 0.001         | 0.001        |
| 0.9890          | 0.5      | 0.001         | 0.001        |


The code in the cell below compiles the model using the chosen optimizer with learning rate and weight decay hyperparameters selected. The model is then fit and the history is captured. Run the code in the cell below, which may take some time. 

In [ ]:
## Compile the model
nn.compile(optimizer=RMSprop(learning_rate=0.001, weight_decay=0.001), 
           loss = 'categorical_crossentropy', metrics = ['accuracy', 'top_k_categorical_accuracy'])

## Now fit the model
history = nn.fit(train_images, train_labels, 
                  epochs = 80, batch_size = 256,
                  validation_data = (test_images, test_labels),
                  callbacks = callbacks_list,  # Call backs argument here
                  verbose = 1)

Run the code in the cells below to plot the loss and accuracy history vs. epoch.

In [ ]:
def plot_loss(history):
    '''Function to plot the loss vs. epoch'''
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = list(range(1, len(train_loss) + 1))
    plt.plot(x, val_loss, label = 'Validation loss')
    plt.plot(x, train_loss, color = 'red', label = 'Training losss')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss vs. Epoch')
    plt.show()
    
def plot_accuracy(history):
    x = list(range(1, len(history.history['accuracy']) + 1))
    plt.plot(x, history.history['accuracy'], color = 'red', label = 'Training accuracy')
    plt.plot(x, history.history['val_accuracy'], label = 'Validation accuracy')
    plt.plot(x, history.history['top_k_categorical_accuracy'], label = 'Top 2 accuracy')
    plt.plot(x, history.history['val_top_k_categorical_accuracy'], label = 'Top 2 validation accuracy')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs. Epoch')      
    plt.show()
    
plot_loss(history)    
plot_accuracy(history)   

> **Exercise 4-3:** Examine these results and answer the following questions:    
> 1. How does the accuracy and error rate of your CovNet model compare to the regularized fully connected model of Exercise 3-10 of the previous assignment?
> 2. How can you characterize the convergence or learning rate of the CovNet model after the first 10 epochs.
> 3. Notice that after only a few epochs the top-2 accuracy is nearly 1.0. Given the nature of the output layer of the model, how can you explain this behavior vs. the behavior of the top-1 accuracy? 
>
> **End of Exercise.**

> **Answers:**   
> 1.            
> 2.          
> 3.            

Now you will continue with the evaluation of the model. One can compute several possible accuracy metrics.    

First, you will use **top-1** accuracy along with **top-k** accuracy, in this case top-2. Top-1 accuracy measures the fraction of cases where the correct category is predicted. Top-k accuracy measures the fraction of cases where the correct category is in the top k probabilities of categories. Since there are only 10 categories for this problem we are limiting to $k=2$. For cases with many categories it is common to use top-5 accuracy.    

You will also compute and display the mean average precision and mean average recall.  

Execute this code in the cell below to compute and display these accuracy metrics along with average precision and average recall.    

In [ ]:
def evaluate_model(model, test_imges, test_labels):   
    ## Compute class predictions
    class_probabilities = model.predict(test_images)
    class_predictions = class_probabilities.argmax(axis=1)
    
    ## Compute the top-1 categorical accuacy.  
    categorical_accuracy = CategoricalAccuracy()
    categorical_accuracy.update_state(test_labels, class_probabilities)

    # Compute the top-2 accuracy.  
    top_2_accuracy = TopKCategoricalAccuracy(k=2, name='top_2_categorical_accuracy')
    top_2_accuracy.update_state(test_labels, class_probabilities)

    ## Summarize and print the result.   
    print('Categorical accuracy = ' + str(categorical_accuracy.result().numpy()))
    print('Top 2 accuracy = ' + str(top_2_accuracy.result().numpy()))

    ## Get the class labels, counts and compute the class-specific recall and precision.  
    test_labels_vector = test_labels.argmax(axis=1)
    unique_labels, label_counts = np.unique(test_labels_vector, return_counts=True)
    class_precision = metrics.precision_score(test_labels_vector, class_predictions, labels=unique_labels, average=None)
    class_recall = metrics.recall_score(test_labels_vector, class_predictions, labels=unique_labels, average=None)

    ## Comptue and print the mean average precision and recall accounting for class frequency weights
    sum_label_counts = np.sum(label_counts)
    weighted_average = lambda x: round(np.sum(np.divide(x * label_counts, sum_label_counts)), 4)
    print('Average precision = ' + str(weighted_average(class_precision)))
    print('Average recall = ' + str(weighted_average(class_recall)))

    ## Compute, display and plot the confusion matrix.  
    confusion_matrix = metrics.confusion_matrix(test_labels_vector, class_predictions)   
    print('\nConfusion matrix')
    print(pd.DataFrame(confusion_matrix))
    p = plt.imshow(np.log(np.divide(confusion_matrix + 1.0, np.sum(confusion_matrix, axis=1))))
    cb = plt.colorbar(p)
    _=cb.set_label('Log count')


evaluate_model(nn, test_images, test_labels)

> **Exercise 4-4:** Examine the evaluation metrics of the model. Notice that the accuarcy, mean average precision and mean average recall are all high. Now, answer these questions:    
> 1. Consider the differences between the top-1 (categoracal) accuraccy and top-2 accuracy. Is the difference expected for this model and why?   
> 2. Why are average precision and average recall values computed expected for this model?   

> **Answers:**     
> 1.        
> 2.             

## 3.0 What features does the convolutional neural network learn?

You might well ask, what is in the various layers of the feature map? It turns out, we can actually visualize the output from each layer of the convolutional neural network to get a feel for the learned features. This can be done for both convolutional and max pooling layers.

As a first step we must pick and image used as the test case. For this example, we arbitrarily choose the 13th image in the training tensor. Execute the code in the cell below to visualize this image. 

In [ ]:
img = train_images[12,:,:,:]
print(img.shape)
plt.imshow(img.reshape((28,28)), cmap='gray')

Our sample image turns out to be a number 3. 

The code in cell below does the following:
1. Extracts the layers from the model object into a list. 
2. Creates an activation map for each layer using the original model object and the list of layer outputs. 
3. Computes the activations for each layer by applying the predict method to the activation map.

> **Note:** In some combinations of environment and versions of Keras and TensorFlow, you may need to change the name of the model object attribute, `nn.input` to `nn.inputs`. 

We will now extract the activations from the trained model by the following steps.  

### 3.1 - Extract the layers from the model    

The code below creates a list of the output layers we want to visualize. Execute this code to create the list of these layers.  

In [ ]:
layer_names = [layer.name for layer in nn.layers] # create list of layer names
layer_indx = [0,1,3,4,6] # indices of layers we want to visualize
layer_outputs = [nn.get_layer(layer_names[i]).output for i in layer_indx]
layer_outputs

### 3.2 - Create and compute an activation model    

With the layers we want to visualize the activations defined, we can now build a model and extract the activations. This temporary model inherits the weights already learned for the full model. The activations are computed by applying the predict method to the model, with a correctly shaped image tensor as input. Execute the code in the cell below.     

In [ ]:
activation_model = models.Model(inputs=nn.inputs[0], outputs=layer_outputs)

img_tensor = np.expand_dims(img, axis=0)  # shape becomes (28, 28, 1, )
activations = activation_model.predict(img_tensor)

### 3.3 Display the activations

Now that the activations have been computed for the test image, it is time to display them. The code in the cell below does the following for each layer of interest:    
1. A plot grid is laid out for each channel in the layer.     
2. The activations for each channel of the layer is plotted on the grid.

Execute the code in the cell and examine the resulting activation plots.  

In [ ]:
# Visualize each layer's activations
for layer_name, layer_activation in zip(layer_names, activations):
    # Drop the batch dimension: shape (height, width, channels)
    activation_maps = layer_activation[0]
    num_channels = activation_maps.shape[-1]

    # Compute a grid size so all channels fit in a square
    grid_size = int(math.ceil(math.sqrt(num_channels)))
    
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(10, 10))
    axes = axes.flatten()
    fig.suptitle(layer_name)

    # Plotting each channel
    for i in range(num_channels):
        axes[i].imshow(activation_maps[..., i], cmap='gray')
        axes[i].axis('off')

    for i in range(num_channels, grid_size * grid_size):
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

> **Exercise 4-5:** Examine the above sets of images. There are 5 sets of images, one for each of the output of the first 5 layers of the model. For each layer, there is an image showing each channel of the feature map. Keeping in mind that CNNs are known to be effective as finding find-grain features, in one or a few sentences, answer the following questions: 
> 1. The initial single channel $28 \times 28 \times 1$ input image tensor becomes a $32 \times 26 \times 26 \times 1$, 32 channel feature map after the first convolution. The second $32 \times 13 \times 13 \times 1$ output is from the max-pooling layer. In general terms, describe how much these feature maps resemble the original image, and why do you think this might be the case given the receptive field of the kernels?    
> 2. Examine the feature maps created by the forth (convolution), fifth (max-pooling), and seventh (convolution) layers. How can you best describe the level of abstraction of these layers and the the resemblance to the original image?      
> 3. What effect can you observe of using the max-pooling layer other than dimensionlity reduction, and why?
> 4. How does the 
> 5. What general relationship or trend can you identify between the spatial dimensionality of the feature map and the level of abstraction and localization of the features? 
> **End of exercise**.  

> **Answers:**
> 1.           
> 2.            
> 3.          
> 4.          

## 4.0 Padding convolution

As you have observed, each convolution layer in a neural network reduces the spatial dimensions by $\frac{span+1}{2}$ for odd span kernels. The coverage of the convolution operator can be expanded by **zero padding** in the spatial dimension. With the zero padding added, the convolution operator covers the entire spatial dimension of the input tensor. This type of convolution is sometime referred to as **same convolution**, since the output tensor has the same spatial dimensions. 

In some situations, adding zero padding produces a better feature map. Zero padding allows each pixel to be visited the same number of times by the convolution operator. Without padding pixels at the edge, are not full represented in the feature map. More importantly, the larger feature map will have larger model capacity. 

2-d convolution with zero padding is illustrated in Figure 4.1 below. The original input 2-d input tensor is shown in gray. The span of the kernel is 3X3 so one additional column of zeros is added to each sided along with a row of zeros top and bottom. As illustrated, the kernel can operate on the edge pixels. The result is an output tensor with the same spatial dimension as the input. 

![](img/Padding.JPG)
<center>**Figure 4.1. 2-d convolution with zero padding**</center>

> **Exercise 4-6:** With Keras, adding zero padding to convolutional layers is easy. The `padding = 'same'` argument adds zero padding. The default is `padding = 'valid'` adds no padding. Create a new model, with a new name 'nns', with the same layers as the foregoing model, and a dropout rate of 0.2, but with the `padding = 'same'` argument for the convolutional layers.  Then print the summary of the model.  

In [ ]:
## Your code goes below


nns.summary()

> Examine the summary of your model and answer these questions:   
> 1. The final feature map is $64 \times 7 \times 7$, rather than $64 \times 3 \times 3$ for the model without padding. This feature map has 49 pixels per channel, compared to 9 pixels per channel previously. Do you expect that this feature map will contain more information and why? 
> 2. How has the number of parameters changed for the convolutional layer, and is this expected? 
> 3. Compare the number of learnable parameters for the 64 unit fully connected layer between the models with and without padding. How can you account for this difference?
>    
> **End of exercise**. 

**Answers:**   
> 1.      
> 2.        
> 3.          

Next execute the code in the cell below to compile and fit the model. Notice that we have increased patience from 2 to 5 for the early stopping criteria. This process will take some time. 

In [ ]:
## Compile the model
nns.compile(optimizer=RMSprop(learning_rate=0.001, weight_decay=0.001), 
           loss = 'categorical_crossentropy', metrics = ['accuracy', 'top_k_categorical_accuracy'])

## Now fit the model
patience = 5
callbacks_list = set_callbacks_list(patience, filepath)
history_nns = nns.fit(train_images, train_labels, 
                  epochs = 80, batch_size = 256,
                  validation_data = (test_images, test_labels),
                  callbacks = callbacks_list,  # Call backs argument here
                  verbose = 1)

Next, execute the code in the cells below to plot the loss vs. epoch and accuracy vs. epoch.

In [ ]:
plot_loss(history_nns)    
plot_accuracy(history_nns) 

In [ ]:
## Display accuracy netrics
evaluate_model(nns, test_images, test_labels)

> **Exercise 4-7:** Keeping in mind that small (3rd digit) differences in metrics are often simply a result of random sampling do these results appear significantly difference than for the un-padded model? It may well be the case that richer feature map may not be important for this relatively simple problem. 

> **Answer:**            

## A More Efficient CNN

As you have likely noticed, training the foregoing CNN models is a rather computationally intensive process. This was even the case got the small networks used for these examples. In large part, the slow training is a result of the large number of parameters that must be learned for 4-D and higher dimensional convolutions. Consider that for a 32 to 32 channel $3 \times 3$, 4-D convolution operator the network must learn $3 \times 3 x 32 x 32 = 9,216$ parameters.       

There is a more efficient formulation of the CNN layer, the **separable convolution operator**. The separable convolution operator consists of two components, a **depth-wise 2-D convolution operator** and a **point-wise convolution operator**. The depth-wise convolution learns a set of 2-D weights independently for each channel. The point-wise convolution learns the weights of a 1-D convolution operator that 'mixes between the channels'. Separating the convolutional operator like this greatly reduces the number of parameters that must be learned. For a $3 \times 3$ 32 to 32 channel convolution the separable operator has $3 \ times 3 + 32 + 32 = 73$ parameters, a large change compared to the 4-D operator.    

> **Exercise 4-8:** To see the the result of applying separable convolution to the digit classification problem you will do the following. You will use a different method to specify a sequential Keras model than you have used previously. The layers are specified in evaluation order within the list which is the argument to the [keras.models.sequntial](https://keras.io/guides/sequential_model/) function. To complete the code in the function below you can copy the layers already used for Exercise 6-6. You only need to modify the code that defines the convolutional layers, not use the `add` method on the model. Make sure you put comments between the layer specifications in the list. To help you get started the specification of the first two layers is provided. When you have completed the layer list, execute the code in the cell below.                      

In [ ]:
def build_sep_cnn(shape=(28, 28, 1, ), dropout=0.2, padding='same'):
    cnnsep = models.Sequential(
        [
           layers.Input(shape=shape),
           layers.SeparableConv2D(32, (3, 3), padding=padding, activation = 'relu'),
        
           ## Put the layer list here    
           


            
        ]
    )
    return cnnsep

nnsep = build_sep_cnn()
nnsep.summary()

In [ ]:
## Compile the model
nnsep.compile(optimizer=RMSprop(learning_rate=0.001, weight_decay=0.001), 
           loss = 'categorical_crossentropy', metrics = ['accuracy', 'top_k_categorical_accuracy'])

## Now fit the model
patience = 5
callbacks_list = set_callbacks_list(patience, filepath)
history_nnsep = nnsep.fit(train_images, train_labels, 
                  epochs = 80, batch_size = 256,
                  validation_data = (test_images, test_labels),
                  callbacks = callbacks_list,  # Call backs argument here
                  verbose = 1)

## Display learning history
plot_loss(history_nnsep)    
plot_accuracy(history_nnsep) 

## Display accuracy netrics
evaluate_model(nnsep, test_images, test_labels)

> Examine these results and answer the following questions.      
> 1. How do the performance metrics of the model using the separable convolution compare the the model you constructed and evaluated in Exercise 4-7?
> 2. Compare the number of model parameters of the separable convolution layers to the full convolution layers in the model you created in Exercise 4-6. What does this difference tell you about separable convolution vs. full convolution?      
> 3. Compare the time required per step to train the model with separable convolution to the time per step for the fully convolution model. Given the reduced number of parameters, is the difference in time required consistent?       

> **Answers:**
> 1.        
> 2.         
> 3.            

## Variable Learning Rate     

The two foregoing models have been trained using a fixed learning rate. In many cases a fixed learning rate is far from optimal. The problem arises since initial learning can take large steps, but small steps are required toward convergence to ensure a stable termination of the algorithm. A fixed low learning rate will learn too slowing a the beginning of training but converge at the end. On the other hard a high learning rate is effective at the beginning of training, but will lead to poor or unstable convergence.   

The solution is to use a variable learning rate. The variable learning rate decreases at the training proceeds. In this way, the learning rate can be optimized to ensure the fastest possible convergence of the learning. The variable learning rate is determined by a schedule. Common schedules include linear decreases and exponential decreases. Keras supports a number of [learning rate schedulers](https://keras.io/api/optimizers/learning_rate_schedules/).      

> **Exercise 4-9:** You will now investigate using a variable learning rate for training the CNN model. The code in the cell bellow does the following:  
> 1. An exponentially decay learning rate schedule is created using [tensorflow.keras.optimizers.schedules.ExponentialDecay](https://keras.io/api/optimizers/learning_rate_schedules/exponential_decay/). Notice that the initial learning rate has been increased from the previous exercise.  
> 2. The `nnv` model is instantiated, using the function you created earlier, and compiled. A learning rate schedule can be incorporated into most Keras optimizers. In this case, the [RMSprop optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop) has an `learning_rate` argument set to the learning rate just created, and with `weight_decay=0.001`.    
> 3. The model is fit using 80 epochs, or until the stopping criteria set in the call-back function is reached. The larger number of epochs allow the training to run closer to completion.    
> 
> Execute this code to training the model, which will take some time to run.

> **Note:** You may be wondering why a new model needs to be instantiated and compiled to run this experiment. The reason is that Keras (and TensorFlow) model objects will retain any weight values from previous learning. This property can be an advantage if one wishes to do additional training. For example, additional training can be performed when additional data is available, without starting over again. However, if one wishes to run an independent experiment, as we do here, it is best to instantiate a new model object.    

In [ ]:
## Define the learning rate schedule 
initial_learning_rate = 0.005
lr_schedule = ExponentialDecay(
    initial_learning_rate, 
    decay_steps=100,
    decay_rate=0.96,
    staircase=True)

## Compile the model
nnv = build_sep_cnn()
nnv.compile(optimizer = RMSprop(learning_rate=lr_schedule, weight_decay=0.001),
                loss = 'categorical_crossentropy', 
                metrics = ['accuracy', 'top_k_categorical_accuracy'])
    
## Now fit the model
patience = 5
callbacks_list = set_callbacks_list(patience, filepath)
history_vnnsep = nnv.fit(train_images, train_labels, 
                  epochs = 80, batch_size = 256,
                  validation_data = (test_images, test_labels),
                  callbacks = callbacks_list,  # Call backs argument here
                  verbose = 1)

> Next execute the code in the cell below to evaluate the model just trained.   

In [ ]:
plot_loss(history_vnnsep)    
plot_accuracy(history_vnnsep) 

## Display accuracy metrics
evaluate_model(nnsep, test_images, test_labels)   

> Examine the results and compare them to those created for Exercise 6-3.      
> 1. Compare the number of epochs that occur before reaching either the early stopping criteria or the limit with the model trained in Exercise 4-8. What does this difference tell you about the effect of using a decaying learning rate?    
> 2. Notice the difference in the gap between the training and validation curves for both loss and top-1 accuracy compared to the model trained in Exercise 4-8. What does this difference tell you about the training and generalization of the respective models?   

> **Answers:**   
> 1.           
> 2.        

## Model with Residual Blocks

To train very deep models one needs to do more than just stack sequentially evaluated convolutional layers. Simply stacking increasing numbers of convolutional layers results in diminishing gradients during back propagation. In this case, we are working with relatively shallow networks, which should not encounter problems with vanishing gradients. None the less, we can demonstrate the principle.    

The code below implements a model with two residual blocks. This code uses another method for specifying Keras models. The output of each layer is set to a named variable. The input of each layer is explicitly specified using these named variables. Using this approach allows us to create non-sequential models. In this case, we must use an addition operation to combine the residual from the convoluton layers with the **identity shortcut** branch.    

The residual blocks use separable convolutional layers. The first residual block has 32 channels and the second residual block has 64 channels. A $1 \times 1$ convolutional layer is used to transform the 32 channel identity shortcut from the first residual block to the required 64 channels for the second residual block. Each residual block is followed by a max pooling layer.    

Execute the code in the cell below and examine the model summary.  

In [ ]:
CNN_input = layers.Input(shape=(28, 28, 1, ))
u = layers.SeparableConv2D(32, (3, 3), padding = 'same', activation = 'relu')(CNN_input)

## A residual block
v = layers.SeparableConv2D(32, (3, 3), padding = 'same', activation = 'relu')(u)
v = layers.SeparableConv2D(32, (3, 3), padding = 'same', activation = 'relu')(v)
v = layers.Add()([v,u]) # Combine the output of the pervious layer with the output of the convolutional layers
v = layers.MaxPooling2D((2, 2))(v)

## Another residual block
r = layers.Conv2D(filters=64,kernel_size=(1,1))(v) # 1x1 convolution layers to trasform form 32 to 64 channels
w = layers.SeparableConv2D(64, (3, 3), padding = 'same', activation = 'relu')(v)
w = layers.SeparableConv2D(64, (3, 3), padding = 'same', activation = 'relu')(w)
w = layers.Add()([w,r]) # Combine the output of the pervious layer with the output of the convolutional layers
w = layers.MaxPooling2D((2, 2))(w)

## Flatten and classify
x = layers.Flatten()(w)
x= layers.Dense(64, activation = 'relu',
                        kernel_regularizer=regularizers.l2(0.1))(x)
x = Dropout(0.5)(x)
CNN_output = layers.Dense(10, activation = 'softmax')(x)

CNN_model = models.Model(CNN_input, CNN_output)
CNN_model.summary()

While this model has an additional separable convolution layer the total number of parameters that must be learned has increased only slightly compared to the sequential model we have been working with.     

Since the model structure is not sequential, it will be useful to visualize how the layers are connected. Display a graph of the model by executing the code in the cell below which uses the [keras.utils.plot_model](https://keras.io/api/utils/model_plotting_utils/) function. 

In [ ]:
plot_model(CNN_model, "mini_resnet.png", show_shapes=True, dpi=60)

With the model defined it is time to train and evaluate it. The same process and hyperparameters are used here. Execute the code below and examine the results. 

In [ ]:
## Define the learning rate schedule 
initial_learning_rate = 0.005
lr_schedule = ExponentialDecay(
    initial_learning_rate, 
    decay_steps=100,
    decay_rate=0.96,
    staircase=True)

## compile the model
CNN_model.compile(optimizer = RMSprop(learning_rate=lr_schedule, weight_decay=0.01),
                loss = 'categorical_crossentropy', 
                metrics = ['accuracy', 'top_k_categorical_accuracy'])
    
## Now fit the model
patience = 5
callbacks_list = set_callbacks_list(patience, filepath)
history_CNN_model = CNN_model.fit(train_images, train_labels, 
                  epochs = 80, batch_size = 128,
                  validation_data = (test_images, test_labels),
                  callbacks = callbacks_list,  # Call backs argument here
                  verbose = 1)

plot_loss(history_CNN_model)    
plot_accuracy(history_CNN_model) 

## Display accuracy netrics
evaluate_model(CNN_model, test_images, test_labels)   

> **Exercise 4-10:** Answer the following questions based on the results shown above.    
> 1. Are the accuracy results of the model with residual blocks comparable to the sequential model evaluated in Exercise 6-9 and why should this result be expected.       
> 2. Is there a significant difference in the number of training epochs required between the sequential model used in Exercise 6-9 and the model with residual blocks and why would this behavior be expected?    
> 3. Why is a $1 \times 1$ convolution layer required in the identity loop of the second residual block.

> **Answers:**:
> 1.        
> 2.          
> 3.       

## Summary

In these exercises you have applied convolutional neural networks (CNNs) to an image calassification problem. Some key points of this lesson are:     
1. CNNs learn powerful features from images.
2. Convolutional feature maps have multiple channels with a spatial dimensions.      
3. Multi-layer CNNs create highly absract features at deeper layers. Deeper architectures can learn more powerful features.
4. The convolutional feature map is used as an input to a classifier.
5. Evaluation of multi-class classifiers can be evaluated by both top-1 and top-k accuacy metrics.   

#### Copyright  2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, Stephen F Elston. All rights reserved.